# PulmonarIA

## Dependências

In [14]:
from functools import partial
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import seaborn as sns
import os

## Diretório do dataset

In [15]:
data_dir = "/kaggle/input/chest-xray-pneumonia/chest_xray/"
train_dir = os.path.join(data_dir, "train")
val_dir = os.path.join(data_dir, "val")
test_dir = os.path.join(data_dir, "test")

## Data augmentation e tratamento das imagens

In [16]:
train_data_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,  
    zoom_range=0.2, 
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_data_gen.flow_from_directory(
    train_dir, 
    target_size=(150, 150), 
    batch_size=32, 
    class_mode='binary',
    color_mode='grayscale'
)

data_gen = ImageDataGenerator(rescale=1./255)

val_generator = data_gen.flow_from_directory(
    val_dir, 
    target_size=(150, 150), 
    batch_size=32, 
    class_mode='binary',
    color_mode='grayscale'
)

test_generator = data_gen.flow_from_directory(
    test_dir, 
    target_size=(150, 150), 
    batch_size=32, 
    class_mode='binary', 
    shuffle=False,
    color_mode='grayscale'
)

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


## Construção do modelo

In [17]:
DefaultConv2D = partial(Conv2D, kernel_size=3, activation='relu', padding="SAME")

model = Sequential([
    BatchNormalization(input_shape=[150, 150, 1]), 
    DefaultConv2D(filters=64, kernel_size=3),
    MaxPooling2D(2, 2),
    DefaultConv2D(filters=128),
    DefaultConv2D(filters=128),
    MaxPooling2D(2, 2),
    DefaultConv2D(filters=256),
    DefaultConv2D(filters=256),
    MaxPooling2D(2, 2),
    GlobalAveragePooling2D(),
    Dense(units=128, activation='relu'),
    Dropout(0.5),
    Dense(units=64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/normalization/batch_normalization.py:143: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


## Treino do modelo

In [18]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [19]:
history = model.fit(train_generator, validation_data=val_generator, epochs=15)

Epoch 1/15


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


163/163 ━━━━━━━━━━━━━━━━━━━━ 39s 202ms/step - accuracy: 0.7286 - loss: 0.6168 - val_accuracy: 0.5000 - val_loss: 0.7607
Epoch 2/15
163/163 ━━━━━━━━━━━━━━━━━━━━ 33s 196ms/step - accuracy: 0.7372 - loss: 0.5783 - val_accuracy: 0.5625 - val_loss: 1.4816
Epoch 3/15
163/163 ━━━━━━━━━━━━━━━━━━━━ 34s 199ms/step - accuracy: 0.8065 - loss: 0.3682 - val_accuracy: 0.5625 - val_loss: 1.7430
Epoch 4/15
163/163 ━━━━━━━━━━━━━━━━━━━━ 34s 197ms/step - accuracy: 0.9001 - loss: 0.2414 - val_accuracy: 0.7500 - val_loss: 0.6490
Epoch 5/15
163/163 ━━━━━━━━━━━━━━━━━━━━ 34s 198ms/step - accuracy: 0.9405 - loss: 0.1719 - val_accuracy: 0.5000 - val_loss: 1.9424
Epoch 6/15
163/163 ━━━━━━━━━━━━━━━━━━━━ 33s 194ms/step - accuracy: 0.9448 - loss: 0.1486 - val_accuracy: 0.6250 - val_loss: 0.7695
Epoch 7/15
163/163 ━━━━━━━━━━━━━━━━━━━━ 34s 200ms/step - accuracy: 0.9471 - loss: 0.1422 - val_accuracy: 0.6875 - val_loss: 1.0309
Epoch 8/15
163/163 ━━━━━━━━━━━━━━━━━━━━ 34s 197ms/step - accuracy: 0.9582 - loss: 0.1174 - val

In [ ]:
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc:.4f}")

## Validação do modelo (threshold 0.6)

In [ ]:
y_pred = (model.predict(test_generator) > 0.6).astype("int32").flatten()
y_true = test_generator.classes

print(classification_report(y_true, y_pred, target_names=["NORMAL", "PNEUMONIA"]))

## Matriz de confusão do modelo

In [ ]:
cm = confusion_matrix(y_true, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=["NORMAL", "PNEUMONIA"], yticklabels=["NORMAL", "PNEUMONIA"])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Matriz de Confusão (Threshold 0.6)")
plt.show()